1. [Introduction](#Introduction)
2. [Dataset](#Dataset)
3. [Preprocessing the Dataset](#Preprocessing)
4. [Setup Parameters for Model Training](#Setup-Parameters-for-Model-Training)
5. [Model Training](#Model-Training)
6. [Deploy SageMaker Model](#Deploy-SageMaker-model)
7. [Setup Clarify Job Parameters](#Set-up-Clarify-config)
8. [Run Clarify Explainability](#Run-Clarify-Explainability)
9. [Evaluate Outputs](#Evaluate-Output)


# Introduction

### This notebook  will prepare the dataset and model for the model explainability lab.  This is an optional step if you have kept your artifacts from previous modules.

### Import modules and initialize parameters for this notebook

In [ ]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
sess = sagemaker.Session()

account = sess.account_id()
region = sess.boto_region_name
bucket = sess.default_bucket() # or use your own custom bucket name
prefix = 'clarify-explainability'

# Dataset

### The dataset we are using is from [Caltech Birds (CUB 200 2011)](http://www.vision.caltech.edu/visipedia/CUB-200-2011.html) dataset contains 11,788 images across 200 bird species (the original technical report can be found here). Each species comes with around 60 images, with a typical size of about 350 pixels by 500 pixels. Bounding boxes are provided, as are annotations of bird parts. A recommended train/test split is given, but image size data is not.

### Run the cell below to download the full dataset or download manually [here](https://course.fast.ai/datasets). Note that the file size is around 1.2 GB, and can take a while to download.If you plan to complete the entire workshop, please keep the file to avoid re-download and re-process the data.

### Downloading the dataset and Uploading the images to S3 takes approx 10-15 minutes

# Preprocessing

In [ ]:
!wget 'https://s3.amazonaws.com/fast-ai-imageclas/CUB_200_2011.tgz'
!tar xopf CUB_200_2011.tgz
!rm CUB_200_2011.tgz

s3_raw_data = f's3://{bucket}/{prefix}/full/data'
!aws s3 cp --recursive ./CUB_200_2011 $s3_raw_data
!rm -rf ./CUB_200_2011

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
)
import time 

timpstamp = str(time.time()).split('.')[0]
# SKlearnProcessor for preprocessing
output_prefix = f'{prefix}/outputs'
output_s3_uri = f's3://{bucket}/{output_prefix}'

class_selection = '13, 17, 35, 36, 47, 68, 73, 87'
input_annotation = 'classes.txt'
processing_instance_type = "ml.m5.xlarge"
processing_instance_count = 1

sklearn_processor = SKLearnProcessor(base_job_name = f"{prefix}-preprocess",  # choose any name
                                    framework_version='0.20.0',
                                    role=role,
                                    instance_type=processing_instance_type,
                                    instance_count=processing_instance_count)

### The next step takes 7-10 minutes to run

In [ ]:
sklearn_processor.run(
    code='preprocessing.py',
    arguments=["--classes", class_selection, 
               "--input-data", input_annotation],
    inputs=[ProcessingInput(source=s3_raw_data, 
            destination="/opt/ml/processing/input")],
    outputs=[
            ProcessingOutput(source="/opt/ml/processing/output/train", destination = output_s3_uri +'/train'),
            ProcessingOutput(source="/opt/ml/processing/output/valid", destination = output_s3_uri +'/valid'),
            ProcessingOutput(source="/opt/ml/processing/output/test", destination = output_s3_uri +'/test'),
            ProcessingOutput(source="/opt/ml/processing/output/manifest", destination = output_s3_uri +'/manifest'),
        ],
    )

### This is where your images and annotation files are located.  You will need these for this module.

In [ ]:
print(f"Test dataset located here: {output_s3_uri +'/test'} ===========")

print(f"Test annotation file is located here: {output_s3_uri +'/manifest'} ===========")

# Setup Parameters for Model Training

In [ ]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.tensorflow import TensorFlow

TF_FRAMEWORK_VERSION = '2.1'

hyperparameters = {'initial_epochs':     5,
                   'batch_size':         8,
                   'fine_tuning_epochs': 20, 
                   'dropout':            0.4,
                   'data_dir':           '/opt/ml/input/data'}

metric_definitions = [{'Name': 'loss',      'Regex': 'loss: ([0-9\\.]+)'},
                  {'Name': 'acc',       'Regex': 'accuracy: ([0-9\\.]+)'},
                  {'Name': 'val_loss',  'Regex': 'val_loss: ([0-9\\.]+)'},
                  {'Name': 'val_acc',   'Regex': 'val_accuracy: ([0-9\\.]+)'}]


distribution = {'parameter_server': {'enabled': False}}
DISTRIBUTION_MODE = 'FullyReplicated'
    
train_in = TrainingInput(s3_data=output_s3_uri +'/train', distribution=DISTRIBUTION_MODE)
val_in   = TrainingInput(s3_data=output_s3_uri +'/valid', distribution=DISTRIBUTION_MODE)
test_in  = TrainingInput(s3_data=output_s3_uri +'/test', distribution=DISTRIBUTION_MODE)

inputs = {'train':train_in, 'test': test_in, 'validation': val_in}

training_instance_type = 'ml.c5.4xlarge'

training_instance_count = 1

In [ ]:
model_path = f"s3://{bucket}/{prefix}"

estimator = TensorFlow(entry_point='train-mobilenet.py',
               source_dir='code',
               output_path=model_path,
               instance_type=training_instance_type,
               instance_count=training_instance_count,
               distribution=distribution,
               hyperparameters=hyperparameters,
               metric_definitions=metric_definitions,
               role=role,
               framework_version=TF_FRAMEWORK_VERSION, 
               py_version='py3',
               base_job_name=prefix,
               script_mode=True)

# Model Training
### Train the model - This step takes 12-15 minutes to complete

In [ ]:
estimator.fit(inputs)

# Deploy SageMaker model

In [ ]:
from sagemaker.tensorflow import TensorFlowModel
from time import gmtime, strftime
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer


timestamp_suffix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_name = f"{prefix}-classification-model-{timestamp_suffix}"


serializer = IdentitySerializer(content_type="image/jpeg")
deserializer = JSONDeserializer(accept='application/json')

training_job_name = estimator.latest_training_job.name
model_artifacts = f'{model_path}/{training_job_name}/output/model.tar.gz'

model = TensorFlowModel(name=model_name,
              model_data=model_artifacts,
              source_dir='code',
              entry_point='inference.py',
              role=sagemaker.get_execution_role(),
              framework_version=TF_FRAMEWORK_VERSION,
              sagemaker_session=sess)

predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m5.xlarge',
                         serializer=serializer,
                         deserializer = deserializer)



### Download a single image to validate model prediction

In [ ]:
import boto3

BUCKET_NAME = f'{bucket}'
BUCKET_FILE_NAME = 'clarify-explainability/outputs/test/017.Cardinal/Cardinal_0001_17057.jpg'
LOCAL_FILE_NAME = 'Cardinal_0001_17057.jpg'


s3 = boto3.client('s3')
s3.download_file(BUCKET_NAME, BUCKET_FILE_NAME, LOCAL_FILE_NAME)

### Wait till you see the downloaded image in your local directory
### The output class prediction of model should be 017.Cardinal

In [ ]:
from numpy import argmax

possible_classes = ['013.Bobolink',
                    '017.Cardinal',
                    '035.Purple_Finch',
                    '036.Northern_Flicker',
                    '047.American_Goldfinch',
                    '068.Ruby_throated_Hummingbird',
                    '073.Blue_Jay',
                    '087.Mallard']

fn = 'Cardinal_0001_17057.jpg'

with open(fn, 'rb') as img:
    f = img.read()
    x = bytearray(f)
    results = predictor.predict(x)
    predicted_class_idx = argmax(results)
    predicted_class = possible_classes[predicted_class_idx]

predicted_class


In [ ]:
import boto3
import random

s3_client = boto3.client('s3')


objects = s3_client.list_objects_v2(Bucket=bucket, Prefix=f'{output_prefix}/test/')

for obj in objects['Contents']:
    rnd_num = random.randint(1, 10)
    
    if rnd_num == 1:
        filename =obj['Key'].split('/')[-1]
        copy_source = {
            'Bucket': bucket,
            'Key': obj['Key']
        }
        print (filename)
        print (obj['Key'])
        print (bucket)
        s3_client.copy(copy_source, bucket, f'{prefix}/clarify-images/{filename}')
        s3_client.download_file(bucket, obj['Key'], f'{filename}')

In [ ]:
object_categories = class_selection.split(', ')

# Set up Clarify config

### NOTE: We are using only a subset of images for Clarify to speed up processing. If you would like to use the entire dataset please update the s3_data_input_path to the following in the next cell
### s3_data_input_path = f's3://{bucket}/{prefix}/clarify-images'

### Setting up Clarify Job parameters for Image Classification in the next cell

In [ ]:
from sagemaker import clarify

s3_data_input_path = f's3://{bucket}/{output_prefix}/test/017.Cardinal'
clarify_output_prefix = f"{prefix}/cv_analysis_result"
analysis_result_path = "s3://{}/{}".format(bucket, clarify_output_prefix)
explainability_data_config = clarify.DataConfig(
    s3_data_input_path=s3_data_input_path,
    s3_output_path=analysis_result_path,
    dataset_type="application/x-image",
)

image_config = clarify.ImageConfig(
    model_type="IMAGE_CLASSIFICATION", num_segments=20, segment_compactness=5
)

model_config = clarify.ModelConfig(
    model_name=model_name, instance_type="ml.m5.xlarge", instance_count=1,content_type="image/jpeg"
)

shap_config = clarify.SHAPConfig(num_samples=500, image_config=image_config)
predictions_config = clarify.ModelPredictedLabelConfig(label_headers=object_categories)

In [ ]:
import os

account_id = os.getenv("AWS_ACCOUNT_ID", "<your-account-id>")
sagemaker_iam_role = "<AmazonSageMaker-ExecutionRole>"

# Fetch the IAM role to initialize the sagemaker processing job
try:
    role = sagemaker.get_execution_role()
except ValueError as e:
    print(e)
    role = f"arn:aws:iam::{account_id}:role/{sagemaker_iam_role}"

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.m5.xlarge", sagemaker_session=sess
)

# Run Clarify Explainability
### The entire set of images in the Cardinal folder takes appox. 45-50 minutes to complete


In [ ]:
clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
    model_scores=predictions_config,
)

In [ ]:
%%time
output_objects = s3_client.list_objects(Bucket=bucket, Prefix=clarify_output_prefix)
result_images = []

for file_obj in output_objects["Contents"]:
    file_name = os.path.basename(file_obj["Key"])
    if os.path.splitext(file_name)[1] == ".jpeg":
        result_images.append(file_name)

    print(f"Downloading s3://{bucket}/{file_obj['Key']} ...")
    s3_client.download_file(bucket, file_obj["Key"], file_name)

# Evaluate Output
### View the SHAP values and Pixel Heat Maps for the Images in this notebook
### You can also download the report.pdf from your local directory

In [ ]:
from IPython.display import Image

for img in result_images:
    display(Image(img))